 # <span style='color:aliceblue'><center style='background:#000000 border-radius:0px 25px;padding:25px'> 🍃🍄🍁Part 4. Matrix and Vector Computations🌱🌼🍂</center></span>

In [5]:
!pip install numpy

In [6]:
!pip install numexpr

   ---------------------------------------- 0.0/96.6 kB ? eta -:--:--
   ------------ --------------------------- 30.7/96.6 kB 660.6 kB/s eta 0:00:01
   ------------ --------------------------- 30.7/96.6 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 96.6/96.6 kB 690.6 kB/s eta 0:00:00


In [1]:
import numpy as np
from random import uniform

class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel

class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles

    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)

        for _ in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.ang_vel * v_x
                d_y = timestep * p.ang_vel * v_y
                p.x += d_x
                p.y += d_y

    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])

        for _ in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * ang_vel_i[:, np.newaxis] * v_i
            r_i += d_i

        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

def benchmark(npart=100, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)

    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)

# Example of benchmarking
if __name__ == "__main__":
    from timeit import timeit

    print("Benchmarking with 100 particles:")
    print("Python version:", timeit("benchmark(100, 'python')", globals=globals(), number=1), "seconds")
    print("NumPy version:", timeit("benchmark(100, 'numpy')", globals=globals(), number=1), "seconds")


Benchmarking with 100 particles:
Python version: 1.5284464000724256 seconds
NumPy version: 0.44523170008324087 seconds


In [12]:
import numpy as np
from random import uniform
import numexpr as ne
 

class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel
class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles
    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.ang_vel * v_x
                d_y = timestep * p.ang_vel * v_y
                p.x += d_x
                p.y += d_y
    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * ang_vel_i[:, np.newaxis] * v_i
            r_i += d_i
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]
 
    def evolve_numexpr(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])
 
        for i in range(nsteps):
            norm_i = ne.evaluate("sqrt((r_i ** 2).sum(axis=1))")
            v_i = r_i[:, [1, 0]]
            v_i[:, 0] = ne.evaluate("-v_i[:, 0]")
            v_i = ne.evaluate("v_i / norm_i[:, None]")
            d_i = ne.evaluate("timestep * ang_vel_i[:, None] * v_i")
            r_i = ne.evaluate("r_i + d_i")
 
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]
 
def benchmark(npart=100000, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)
    elif method == 'numexpr':
        simulator.evolve_numpy(0.1)
 
# Example of benchmarking
if __name__ == "__main__":
    from timeit import timeit
    print("Benchmarking with 100000 particles:")
    print("Python version:", timeit("benchmark(100000, 'python')", globals=globals(), number=1), "seconds")
    print("NumPy version:", timeit("benchmark(100000, 'numpy')", globals=globals(), number=1), "seconds")
    print("NumExpr version:", timeit("benchmark(100000, 'numexpr')", globals=globals(), number=1), "seconds")

Benchmarking with 100000 particles:
Python version: 2036.057856999978 seconds
NumPy version: 152.32222770003136 seconds
NumExpr version: 131.13930440001423 seconds
